<a href="https://colab.research.google.com/github/DedeYusufK/NN-model-for-Daily-Max-Temperature/blob/main/neural_network_model_using_the_Daily_Max_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================================

# Build and train a neural network model using the Daily Max Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.2 on the normalized dataset.
# ============================================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [2]:
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('loss') < 0.08:
                print("\nReached desired calbacks!")

                # Stop training once the above condition is met
                self.model.stop_training = True

callbacks = myCallback()

In [3]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [4]:
def solution_B5():
    data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
    urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

    time_step = []
    temps = []

    with open('daily-max-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(float(row[1]))
            time_step.append(row[0])
            step=step + 1

    # series=# YOUR CODE HERE
    series = np.array(temps).astype(float)

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=2500

    time_train=  time[:split_time]
    x_train=  series[:split_time]
    time_valid=  time[split_time:]
    x_valid=  series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    model=tf.keras.models.Sequential([
         tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    model.compile(loss='mae', optimizer='adam', metrics='mae')
    model.fit(train_set, batch_size=256, epochs=100)

    # YOUR CODE HERE
    return model

In [ ]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B5()
    model.save("model_B5.h5")

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/100
10/10 [==============================] - 19s 549ms/step - loss: 0.1737 - mae: 0.1737
Epoch 2/100
10/10 [==============================] - 5s 435ms/step - loss: 0.0995 - mae: 0.0995
Epoch 3/100
10/10 [==============================] - 3s 282ms/step - loss: 0.0930 - mae: 0.0930
Epoch 4/100
10/10 [==============================] - 3s 314ms/step - loss: 0.0904 - mae: 0.0904
Epoch 5/100
10/10 [==============================] - 4s 298ms/step - loss: 0.0866 - mae: 0.0866
Epoch 6/100
10/10 [==============================] - 3s 287ms/step - loss: 0.0841 - mae: 0.0841
Epoch 7/100
10/10 [==============================] - 4s 345ms/step - loss: 0.0823 - mae: 0.0823
Epoch 8/100
10/10 [==============================] - 3s 279ms/step - loss: 0.0810 - mae: 0.0810
Epoch 9/100
10/10 [==============================] - 3s 292ms/